In [1]:
import os
import sys
os.chdir('../')
sys.path.append('../')

import torch
from utils.utils import train,prepare,evaluate,tune
from models.Interactors import FIM_Interactor, KNRM_Interactor
from models.SFI import SFI_gating, SFI_gating_MultiView
from configs.ManualConfig import hparams

In [2]:
name='sfi'
hparams['k'] = 30
hparams['select'] = 'gating'
# hparams['threshold'] = 0.5

In [3]:
# hparams['validate'] = True
vocab, loaders = prepare(hparams, pin_memory=False)

[2021-04-17 11:43:46,797] INFO (root) Hyper Parameters are
{'scale': 'demo', 'mode': 'train', 'batch_size': 10, 'title_size': 20, 'abs_size': 40, 'his_size': 50, 'npratio': 4, 'dropout_p': 0.2, 'query_dim': 200, 'embedding_dim': 300, 'filter_num': 150, 'value_dim': 16, 'head_num': 16, 'epochs': 8, 'metrics': 'auc,mean_mrr,ndcg@5,ndcg@10', 'device': 'cpu', 'attrs': ['title'], 'k': 30, 'select': 'gating', 'save_step': [0], 'news_id': False, 'validate': False, 'interval': 10, 'spadam': True}
[2021-04-17 11:43:46,799] INFO (root) preparing dataset...
[2021-04-17 11:43:50,872] INFO (torchtext.vocab) Loading vectors from .vector_cache/glove.840B.300d.txt.pt


In [4]:
record = next(iter(loaders[0]))

In [5]:
from models.Encoders.FIM import FIM_Encoder

encoder = FIM_Encoder(hparams, vocab)
interactor = FIM_Interactor()
# interactor = KNRM_Interactor()

hparams['name'] = '-'.join([name,encoder.name,hparams['select']])

# sfi = SFI_gating_MultiView(hparams, encoder, interactor).to(hparams['device'])
sfi = SFI_gating(hparams, encoder, interactor).to(hparams['device'])

# sfi.load_state_dict(torch.load('data/model_params/sfi-fim-fim-gating/large_epoch2_step16916_[hs=50,topk=50,contra=0,attrs=title].model', map_location=hparams['device'])['model'])

In [6]:
sfi(record)

tensor([[-1.8073, -1.4263, -1.9366, -2.5724, -0.9804],
        [-1.0203, -1.9576, -1.4412, -2.2127, -1.8822],
        [-1.5882, -1.0329, -1.9175, -2.1771, -1.7182],
        [-2.0468, -2.0637, -1.8720, -0.8189, -1.9029],
        [-1.3822, -1.3716, -1.6382, -2.0367, -1.7691],
        [-1.6516, -1.6384, -1.4850, -1.6257, -1.6571],
        [-1.4770, -2.1405, -1.7311, -1.6054, -1.2867],
        [-1.5511, -2.3223, -2.5750, -2.0175, -0.7323],
        [-1.2932, -2.2127, -0.9679, -1.9622, -2.3455],
        [-0.9237, -1.1715, -2.8598, -2.4039, -1.9282]],
       grad_fn=<LogSoftmaxBackward>)

In [ ]:
sfi = tune(sfi, hparams, loaders)

In [ ]:
evaluate(sfi, hparams, loaders[1], loading=True)